<a href="https://colab.research.google.com/github/ililililililililililililililil/INI/blob/ini_pytorch/ini_6_1_Softmax_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [3]:
#For reproducibility
torch.manual_seed(1)

In [4]:
# Discrete Probability Distribution

#이산적인 확률 분포 (<-> 연속적)

In [5]:
# Softmax

#이전 값을 보고 다음에 ~할 확률

z = torch.FloatTensor([1, 2, 3])

In [6]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)
#Softmax: 비율에 따라 합쳐서 1이 되는 결과 (확률)

tensor([0.0900, 0.2447, 0.6652])


In [7]:
hypothesis.sum()

tensor(1.)

In [15]:
#Cross Entropy: 확률분포가 얼마나 비슷한가
#H(P, Q)를 최소화하면 P에 근사하게 됨

In [16]:
# Cross Entropy Loss (Low-Level)

z = torch.rand(3, 5, requires_grad=True)
hypothesis = F.softmax(z, dim=1)
print(hypothesis)
#예측값

tensor([[0.2974, 0.2244, 0.1595, 0.1420, 0.1767],
        [0.1509, 0.1595, 0.1917, 0.2522, 0.2457],
        [0.1002, 0.2226, 0.1878, 0.2624, 0.2271]], grad_fn=<SoftmaxBackward0>)


In [28]:
y = torch.randint(5, (3, )).long()
print(y)
#정답(랜덤)

tensor([1, 2, 2])


In [38]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.]])

In [39]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
print(cost)

tensor(1.6061, grad_fn=<MeanBackward0>)


In [40]:
# Cross Entropy Loss with torch.nn.functional

# Low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.2127, -1.4941, -1.8359, -1.9517, -1.7335],
        [-1.8912, -1.8358, -1.6516, -1.3775, -1.4038],
        [-2.3011, -1.5023, -1.6726, -1.3380, -1.4823]], grad_fn=<LogBackward0>)

In [41]:
# High level
F.log_softmax(z, dim=1)

tensor([[-1.2127, -1.4941, -1.8359, -1.9517, -1.7335],
        [-1.8912, -1.8358, -1.6516, -1.3775, -1.4038],
        [-2.3011, -1.5023, -1.6726, -1.3380, -1.4823]],
       grad_fn=<LogSoftmaxBackward0>)

In [42]:
# Low level
(y_one_hot * -torch.log(F.softmax(z, dim=1))).sum(dim=1).mean() #단순하게

tensor(1.6061, grad_fn=<MeanBackward0>)

In [43]:
# High level
F.nll_loss(F.log_softmax(z, dim=1), y) #단순하게

tensor(1.6061, grad_fn=<NllLossBackward0>)

In [44]:
F.cross_entropy(z, y) #더 단순하게

tensor(1.6061, grad_fn=<NllLossBackward0>)

In [45]:
#Training with Low-level Cross Entropy Loss

x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [49]:
#모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Cost 계산 (1)
  hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) # or .mm or @
  y_one_hot = torch.zeros_like(hypothesis)
  y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
  cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


In [50]:
#Training with F.cross_entropy

#모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # Cost 계산 (2)
  z = x_train.matmul(W) + b # or .nn or @
  cost = F.cross_entropy(z, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 100번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [52]:
# High-level Implementation with nn.Module

class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!
        
    def forward(self, x):
        return self.linear(x)

In [53]:
model = SoftmaxClassifierModel()

In [60]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

  # H(x) 계산
  prediction = model(x_train)

  # Cost 계산
  cost = F.cross_entropy(prediction, y_train)

  # cost로 H(x) 개선
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  # 20번마다 로그 출력
  if epoch % 100 == 0:
    print('Epoch {:4d}/{} Cost: {:6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/1000 Cost: 0.076049
Epoch  100/1000 Cost: 0.074716
Epoch  200/1000 Cost: 0.073428
Epoch  300/1000 Cost: 0.072183
Epoch  400/1000 Cost: 0.070978
Epoch  500/1000 Cost: 0.069812
Epoch  600/1000 Cost: 0.068684
Epoch  700/1000 Cost: 0.067590
Epoch  800/1000 Cost: 0.066530
Epoch  900/1000 Cost: 0.065503
Epoch 1000/1000 Cost: 0.064505
